In [1]:
from serial import Serial
import numpy as np
from time import sleep
from threading import Thread

In [2]:
port = "COM3"
baudrate = 576000
timeout = 10

with Serial(port, baudrate, timeout=timeout) as ser:
    message = np.zeros(7, dtype=np.float32)
    state = np.zeros(6, dtype=np.float32)
    control = np.zeros(14400, dtype=np.float32)
    control[-1] = 3.14
    counter = 0
    sleep(1)
    ser.reset_input_buffer()
    ser.reset_output_buffer()
    ser.flush()
    ser.read_until(b"start")

    while ser.is_open:
        if ser.in_waiting == 0:
            continue
        
        ser.read_until(b"msg")
        message = np.frombuffer(ser.read(message.nbytes), dtype=np.float32)
        time = message[0]
        state = message[1:7]

        if counter % 100 == 0:
            print(f"{counter} Time: {time}, State: {state}")

        if counter % 1000 == 0:
            def write_control():
                ser.write(b"s")
                ser.write(control.tobytes())
            Thread(target=write_control).start()
        counter += 1    

0 Time: 0.029999999329447746, State: [1. 2. 3. 4. 5. 6.]
100 Time: 2.1600000858306885, State: [1. 2. 3. 4. 5. 6.]
200 Time: 3.1600000858306885, State: [1. 2. 3. 4. 5. 6.]
300 Time: 4.159999847412109, State: [1. 2. 3. 4. 5. 6.]
400 Time: 5.159999847412109, State: [1. 2. 3. 4. 5. 6.]
500 Time: 6.159999847412109, State: [1. 2. 3. 4. 5. 6.]
600 Time: 7.159999847412109, State: [1. 2. 3. 4. 5. 6.]


700 Time: 8.15999984741211, State: [1. 2. 3. 4. 5. 6.]
